# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hazm
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, KeyedVectors

## Read data

In [2]:
sanjesh = pd.read_excel('final_not_value.xlsx')
sanjesh.head(5)

,city,state,university,توضیحات,جنسیت مرد,جنسیت زن,نیم سال دوم,نیم سال اول,رشته ها,digits,نوع دوره,نوع پذیرش,گروه آزمایشی
0,تبریز,آذربایجان شرقی,دانشگاه تبریز,NaN,مرد,زن,18,-,برنامه ریزی اجتماعی و تعاون,33601,روزانه,با آزمون,انسانی
1,تبریز,آذربایجان شرقی,دانشگاه تبریز,NaN,مرد,زن,-,21,تاریخ,33602,روزانه,با آزمون,انسانی
2,تبریز,آذربایجان شرقی,دانشگاه تبریز,NaN,مرد,زن,-,24,جامعه شناسی,33603,روزانه,با آزمون,انسانی
3,تبریز,آذربایجان شرقی,دانشگاه تبریز,NaN,مرد,زن,-,36,جغرافیا,33604,روزانه,با آزمون,انسانی
4,تبریز,آذربایجان شرقی,دانشگاه تبریز,NaN,مرد,زن,-,24,حقوق,33605,روزانه,با آزمون,انسانی


In [3]:
kanoon = pd.read_excel('total_kanoon_1401.xlsx')
kanoon.head(5)

,national_rank,quota_rank,region_quota,نوع دوره,دانشگاه,reshte_name,group_name
0,38754,15942,منطقه 1,پيام نور,دانشگاه پیام نور تهران,حسابداری,ریاضی
1,41293,16751,منطقه 1,پيام نور,دانشگاه پیام نور تهران,حسابداری,ریاضی
2,54007,20527,منطقه 2,پيام نور,دانشگاه پیام نور تهران,حسابداری,ریاضی
3,74749,19351,منطقه 3,روزانه,دانشگاه لرستان,علوم ورزشی,ریاضی
4,78696,20509,منطقه 3,روزانه,دانشگاه لرستان,علوم ورزشی,ریاضی


# Normalize the Persian columns in both data

In [4]:
normalizer = Normalizer()

In [5]:
sanjesh['city'] = sanjesh['city'].astype(str)
sanjesh['state'] = sanjesh['state'].astype(str)
sanjesh['university'] = sanjesh['university'].astype(str)
sanjesh['رشته ها'] = sanjesh['رشته ها'].astype(str)
sanjesh['نوع دوره'] = sanjesh['نوع دوره'].astype(str)
sanjesh['گروه آزمایشی'] = sanjesh['گروه آزمایشی'].astype(str)


sanjesh['city'] = sanjesh['city'].apply(normalizer.normalize)
sanjesh['state'] = sanjesh['state'].apply(normalizer.normalize)
sanjesh['university'] = sanjesh['university'].apply(normalizer.normalize)
sanjesh['رشته ها'] = sanjesh['رشته ها'].apply(normalizer.normalize)
sanjesh['نوع دوره'] = sanjesh['نوع دوره'].apply(normalizer.normalize)
sanjesh['گروه آزمایشی'] = sanjesh['گروه آزمایشی'].apply(normalizer.normalize)

In [6]:
kanoon['region_quota'] = kanoon['region_quota'].astype(str)
kanoon['نوع دوره'] = kanoon['نوع دوره'].astype(str)
kanoon['دانشگاه '] = kanoon['دانشگاه '].astype(str)
kanoon['reshte_name'] = kanoon['reshte_name'].astype(str)
kanoon['group_name'] = kanoon['group_name'].astype(str)

# Normalize the selected columns
kanoon['region_quota'] = kanoon['region_quota'].apply(normalizer.normalize)
kanoon['نوع دوره'] = kanoon['نوع دوره'].apply(normalizer.normalize)
kanoon['دانشگاه '] = kanoon['دانشگاه '].apply(normalizer.normalize)
kanoon['reshte_name'] = kanoon['reshte_name'].apply(normalizer.normalize)
kanoon['group_name'] = kanoon['group_name'].apply(normalizer.normalize)


# Find the cosine similarity

In [7]:
vectorizer = TfidfVectorizer()

combined_texts = kanoon['reshte_name'].tolist() + sanjesh['رشته ها'].tolist()
vectorizer.fit(combined_texts)

# Transform the text data
tfidf_matrix_A = vectorizer.transform(kanoon['reshte_name'])
tfidf_matrix_B = vectorizer.transform(sanjesh['رشته ها'])

# Compute cosine similarities
cosine_similarities = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)

# Find the most similar values and their distances
most_similar_values = []
distances = []
    
for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_value = sanjesh['رشته ها'][most_similar_idx]
    similarity_score = row[most_similar_idx]

    # if similarity_score > 0.8:
        # kanoon.at[idx, 'reshte_name'] = most_similar_value
    
    most_similar_values.append(most_similar_value)
    distances.append(similarity_score)

kanoon['Most_Similar_reshte'] = most_similar_values
kanoon['Similarity_reshte'] = distances

In [8]:
combined_texts = kanoon['reshte_name'].tolist() + sanjesh['رشته ها'].tolist()
combined_texts = [text.split() for text in combined_texts] 


model = Word2Vec(sentences=combined_texts, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv

def sentence_embedding(sentence, model):
    words = sentence.split()
    word_vecs = [model[word] for word in words if word in model]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)


kanoon_embeddings = kanoon['reshte_name'].apply(lambda x: sentence_embedding(x, word_vectors))
sanjesh_embeddings = sanjesh['رشته ها'].apply(lambda x: sentence_embedding(x, word_vectors))

# Convert embeddings to numpy arrays
kanoon_embeddings = np.vstack(kanoon_embeddings)
sanjesh_embeddings = np.vstack(sanjesh_embeddings)

# Compute cosine similarities
cosine_similarities = cosine_similarity(kanoon_embeddings, sanjesh_embeddings)

most_similar_values = []
distances = []

for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_value = sanjesh['رشته ها'][most_similar_idx]
    similarity_score = row[most_similar_idx]
    if similarity_score == 1.0 and kanoon.at[idx, 'Similarity_reshte'] >= 0.8:
        kanoon.at[idx, 'reshte_name'] = most_similar_value

    most_similar_values.append(most_similar_value)
    distances.append(similarity_score)

kanoon['Most_Similar_reshte_WE'] = most_similar_values
kanoon['Similarity_reshte_WE'] = distances

In [9]:
vectorizer = TfidfVectorizer()

combined_texts = kanoon['دانشگاه '].tolist() + sanjesh['university'].tolist()
vectorizer.fit(combined_texts)

# Transform the text data
tfidf_matrix_C = vectorizer.transform(kanoon['دانشگاه '])
tfidf_matrix_D = vectorizer.transform(sanjesh['university'])

# Compute cosine similarities
cosine_similari = cosine_similarity(tfidf_matrix_C, tfidf_matrix_D)

# Find the most similar values and their distances
most_similar_values = []
distances = []
for idx, row in enumerate(cosine_similari):
    most_similar_idx = row.argmax()
    most_similar_value = sanjesh['university'][most_similar_idx]
    similarity_score = row[most_similar_idx]

    # if similarity_score > 0.75:
        # kanoon.at[idx, 'دانشگاه '] = most_similar_value
    
    most_similar_values.append(most_similar_value)
    distances.append(similarity_score)




kanoon['Most_Similar_uni'] = most_similar_values
kanoon['Similarity_uni'] = distances

In [10]:
combined_texts = kanoon['دانشگاه '].tolist() + sanjesh['university'].tolist()
combined_texts = [text.split() for text in combined_texts] 


model = Word2Vec(sentences=combined_texts, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv

def sentence_embedding(sentence, model):
    words = sentence.split()
    word_vecs = [model[word] for word in words if word in model]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)


kanoon_embeddings = kanoon['دانشگاه '].apply(lambda x: sentence_embedding(x, word_vectors))
sanjesh_embeddings = sanjesh['university'].apply(lambda x: sentence_embedding(x, word_vectors))

# Convert embeddings to numpy arrays
kanoon_embeddings = np.vstack(kanoon_embeddings)
sanjesh_embeddings = np.vstack(sanjesh_embeddings)

# Compute cosine similarities
cosine_similarities = cosine_similarity(kanoon_embeddings, sanjesh_embeddings)

most_similar_values = []
distances = []

for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_value = sanjesh['university'][most_similar_idx]
    similarity_score = row[most_similar_idx]
    if similarity_score == 1.0 and kanoon.at[idx, 'Similarity_uni'] >= 0.8:
        kanoon.at[idx, 'دانشگاه '] = most_similar_value

    most_similar_values.append(most_similar_value)
    distances.append(similarity_score)

kanoon['Most_Similar_uni_WE'] = most_similar_values
kanoon['Similarity_uni_WE'] = distances

In [11]:
sanjesh["uni_resh"] = sanjesh["رشته ها"] + " " + sanjesh["university"]
kanoon["uni_resh"] = kanoon["reshte_name"] + " " +  kanoon["دانشگاه "]

sanjesh['uni_resh'].fillna(sanjesh['رشته ها'],inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14036\572182085.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sanjesh['uni_resh'].fillna(sanjesh['رشته ها'],inplace=True)


In [12]:
vectorizer = TfidfVectorizer()

combined_texts = kanoon['uni_resh'].tolist() + sanjesh['uni_resh'].tolist()
vectorizer.fit(combined_texts)

# Transform the text data
tfidf_matrix_A = vectorizer.transform(kanoon['uni_resh'])
tfidf_matrix_B = vectorizer.transform(sanjesh['uni_resh'])

# Compute cosine similarities
cosine_similarities = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)

# Find the most similar values and their distances
most_similar_values = []
distances = []

for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_values.append(sanjesh['uni_resh'][most_similar_idx])
    distances.append(row[most_similar_idx])

kanoon['Most_Similar_unireshte'] = most_similar_values
kanoon['Similarity_unireshte'] = distances

In [13]:
combined_texts = kanoon['uni_resh'].tolist() + sanjesh['uni_resh'].tolist()
combined_texts = [text.split() for text in combined_texts] 


model = Word2Vec(sentences=combined_texts, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv

def sentence_embedding(sentence, model):
    words = sentence.split()
    word_vecs = [model[word] for word in words if word in model]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)


kanoon_embeddings = kanoon['uni_resh'].apply(lambda x: sentence_embedding(x, word_vectors))
sanjesh_embeddings = sanjesh['uni_resh'].apply(lambda x: sentence_embedding(x, word_vectors))

# Convert embeddings to numpy arrays
kanoon_embeddings = np.vstack(kanoon_embeddings)
sanjesh_embeddings = np.vstack(sanjesh_embeddings)

# Compute cosine similarities
cosine_similarities = cosine_similarity(kanoon_embeddings, sanjesh_embeddings)

most_similar_values = []
distances = []

for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_value = sanjesh['uni_resh'][most_similar_idx]
    similarity_score = row[most_similar_idx]
    if similarity_score >=0.97  and kanoon.at[idx, 'Similarity_uni'] >= 0.8:
         kanoon.at[idx, 'uni_resh'] = most_similar_value

    most_similar_values.append(most_similar_value)
    distances.append(similarity_score)

kanoon['Most_Similar_unireshte_WE'] = most_similar_values
kanoon['Similarity_unireshte_WE'] = distances

In [14]:
unique_kanoon = set(kanoon['uni_resh'].unique())
unique_sanjesh = set(sanjesh['uni_resh'].unique())

sanjesh_minus_kanoon = unique_sanjesh - unique_kanoon
kanoon_minus_sanjesh = unique_kanoon - unique_sanjesh


# Convert the result back to a list (if needed)
sanjesh_minus_kanoon= list(sanjesh_minus_kanoon)
kanoon_minus_sanjesh= list(kanoon_minus_sanjesh)

In [15]:
print('len of kanoon universities',len(unique_kanoon))
print('len of sanjesh universities',len(unique_sanjesh))
print('len of sanjesh - kanoon universities',len(sanjesh_minus_kanoon))
print('len of kanoon - sanjesh universities',len(kanoon_minus_sanjesh))

len of kanoon universities 2795
len of sanjesh universities 3495
len of sanjesh - kanoon universities 1675
len of kanoon - sanjesh universities 975


In [16]:
hf = pd.DataFrame({'unique_sanjesh': list(unique_sanjesh)})
hf.to_excel('unique_sanjesh.xlsx')
max_len = max(len(sanjesh_minus_kanoon), len(kanoon_minus_sanjesh))

# Step 2: Extend the shorter list with NaN values
sanjesh_minus_kanoon.extend(['gg'] * (max_len - len(sanjesh_minus_kanoon)))
kanoon_minus_sanjesh.extend(['gg'] * (max_len - len(kanoon_minus_sanjesh)))

# Step 3: Create the DataFrame
df = pd.DataFrame({'sanjesh_minus_kanoon': sanjesh_minus_kanoon, 'kanoon_minus_sanjesh': kanoon_minus_sanjesh})


In [17]:
df['kanoon_minus_sanjesh'].fillna('gg',inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14036\3098048229.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['kanoon_minus_sanjesh'].fillna('gg',inplace=True)


In [18]:
df.isna().sum()

sanjesh_minus_kanoon    0
kanoon_minus_sanjesh    0
dtype: int64

In [19]:
df

,sanjesh_minus_kanoon,kanoon_minus_sanjesh
0,علوم و مهندسی صنایع غذایی (این رشته\nمتعلق به ...,راهنمایی و مشاوره دانشگاه شهرکرد
1,دکتری عمومی دامپزشکی دانشگاه آزاد اسلامی استان...,پرستاری دانشگاه علوم پزشکی جندی شاپور_اهواز (ا...
2,مهندسی حرفه‌ای معماری آموزشکده فنی و حرفه‌ای د...,مهندس ایمنی وبازرسی فنی دانشگاه صنعتی شیراز
3,علوم ورزشی دانشگاه سیستان و بلوچستان,آموزش ابتدایی پردیس شهید چمران
4,معماری داخلی موسسه غیرانتفاعی رسام (ویژه خواهر...,کاردرمانی دانشگاه علوم پزشکی جندی‌شاپور اهواز
...,...,...
1670,مهندسی مکانیک بیوسیستم (این رشته\nمتعلق به گرو...,gg
1671,مهندسی نقشه‌برداری دانشگاه بجنورد,gg
1672,مدیریت بازرگانی دانشگاه ارومیه,gg
1673,ضد تروریسم دانشگاه اطلاعات و امنیت ملی امام با...,gg


In [20]:
vectorizer = TfidfVectorizer()

combined_texts = df['sanjesh_minus_kanoon'].tolist() + df['kanoon_minus_sanjesh'].tolist()
vectorizer.fit(combined_texts)

# Transform the text data
tfidf_matrix_A = vectorizer.transform(df['sanjesh_minus_kanoon'])
tfidf_matrix_B = vectorizer.transform(df['kanoon_minus_sanjesh'])

# Compute cosine similarities
cosine_similarities = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)

# Find the most similar values and their distances
most_similar_values = []
distances = []

for idx, row in enumerate(cosine_similarities):
    most_similar_idx = row.argmax()
    most_similar_values.append(df['kanoon_minus_sanjesh'][most_similar_idx])
    distances.append(row[most_similar_idx])

df['Most_Similar'] = most_similar_values
df['Similarity'] = distances

In [21]:
df

,sanjesh_minus_kanoon,kanoon_minus_sanjesh,Most_Similar,Similarity
0,علوم و مهندسی صنایع غذایی (این رشته\nمتعلق به ...,راهنمایی و مشاوره دانشگاه شهرکرد,مهندسی گیاه پزشکی دانشگاه علوم کشاورزی و منابع...,0.562798
1,دکتری عمومی دامپزشکی دانشگاه آزاد اسلامی استان...,پرستاری دانشگاه علوم پزشکی جندی شاپور_اهواز (ا...,پزشکی دانشگاه آزاد اسلامی,0.504730
2,مهندسی حرفه‌ای معماری آموزشکده فنی و حرفه‌ای د...,مهندس ایمنی وبازرسی فنی دانشگاه صنعتی شیراز,مهندسی حرفه‌ای معماری آموزشکده فنی و حرفه‌ای د...,0.794829
3,علوم ورزشی دانشگاه سیستان و بلوچستان,آموزش ابتدایی پردیس شهید چمران,الهیات و معارف اسلامی دانشگاه سیستان و بلوچستان,0.647475
4,معماری داخلی موسسه غیرانتفاعی رسام (ویژه خواهر...,کاردرمانی دانشگاه علوم پزشکی جندی‌شاپور اهواز,معماری داخلی دانشگاه غیرانتفاعی رسام,0.773038
...,...,...,...,...
1670,مهندسی مکانیک بیوسیستم (این رشته\nمتعلق به گرو...,gg,مهندسی مکانیک بیوسیستم (کشاورزی) دانشگاه فردوسی,0.592751
1671,مهندسی نقشه‌برداری دانشگاه بجنورد,gg,مهندسی نقشه‌برداری دانشگاه تبریز (مرند),0.583345
1672,مدیریت بازرگانی دانشگاه ارومیه,gg,الهیات و معارف اسلامی دانشگاه ارومیه,0.394245
1673,ضد تروریسم دانشگاه اطلاعات و امنیت ملی امام با...,gg,آموزش ابتدایی پردیس امام محمد باقر,0.274962


In [25]:
sorted_df = df.sort_values(by='Similarity',ascending=False)
sorted_df[340:400]

,sanjesh_minus_kanoon,kanoon_minus_sanjesh,Most_Similar,Similarity
967,علوم سیاسی دانشگاه آیت اله بروجردی,علوم سیاسی دانشگاه شهرکرد (فارسان),علوم سیاسی دانشگاه آیت‌الله بروجردی,0.754184
1129,تکنولوژی پرتوشناسی دانشگاه علوم پزشکی تبریز,gg,کارشناسی تکنولوژی پرتوشناسی- رادیولوژی دانشگاه...,0.752333
1034,مهندسی بهداشت حرفه‌ای و ایمنی کار دانشکده پیرا...,gg,مهندسی بهداشت حرفه‌ای و ایمنی کار دانشگاه علوم...,0.751396
1660,مهندسی نقشه‌برداری دانشگاه صنعتی خواجه نصیر ال...,gg,مهندسی نقشه‌برداری دانشگاه صنعتی خواجه نصیرالد...,0.751347
1588,تکنولوژی پرتوشناسی دانشگاه علوم پزشکی اصفهان,gg,کارشناسی تکنولوژی پرتوشناسی- رادیولوژی دانشگاه...,0.750758
759,حسابداری آموزشکده فنی و حرفه‌ای پسران اردبیل –...,مددکاری اجتماعی دانشگاه خوارزمی,حسابداری فنی و حرفه‌ای پسران,0.750532
288,مهندسی عمران دانشگاه آیت اله بروجردی,راهنمایی و مشاوره پردیس بنت الهدی صدر,مهندسی عمران دانشگاه آیت‌الله بروجردی,0.750446
1085,مهندسی برق دانشگاه آیت اله بروجردی,gg,مهندسی برق دانشگاه آیت‌الله بروجردی,0.749922
1378,علوم کامپیوتر دانشگاه الزهرا (س) (ویژه خواهران),gg,علوم کامپیوتر دانشگاه الزهرا (س),0.749618
897,دکتری عمومی پزشکی دانشگاه علوم پزشکی قزوین,مهندسی برق مرکز اموزش عالی فیروزاباد,مدیریت دانشگاه علوم پزشکی قزوین,0.749438


In [31]:
sanjesh_no_kanoon = df[df['Similarity'] < 0.72][['sanjesh_minus_kanoon']]

In [32]:
sanjesh_no_kanoon

,sanjesh_minus_kanoon
0,علوم و مهندسی صنایع غذایی (این رشته\nمتعلق به ...
1,دکتری عمومی دامپزشکی دانشگاه آزاد اسلامی استان...
3,علوم ورزشی دانشگاه سیستان و بلوچستان
5,اعضای مصنوعی و وسایل کلامکی دانشگاه علوم پزشکی...
6,پرستاری دانشکده پرستاری جوین)
...,...
1670,مهندسی مکانیک بیوسیستم (این رشته\nمتعلق به گرو...
1671,مهندسی نقشه‌برداری دانشگاه بجنورد
1672,مدیریت بازرگانی دانشگاه ارومیه
1673,ضد تروریسم دانشگاه اطلاعات و امنیت ملی امام با...


In [35]:
sanjesh_no_kanoon.to_excel('sanjesh_minus_kanoon.xlsx',index=False)